In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from model import ModelClass

In [ ]:
def dice_loss(pred, target):
    """This definition generalize to real valued pred and target vector.
This should be differentiable.
    pred: tensor with first dimension as batch
    target: tensor with first dimension as batch
    """

    smooth = 1.

    # have to use contiguous since they may from a torch.view op
    iflat = pred.contiguous().view(-1)
    tflat = target.contiguous().view(-1)
    intersection = (iflat * tflat).sum()

    A_sum = torch.sum(tflat * iflat)
    B_sum = torch.sum(tflat * tflat)
    
    return 1- ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )

In [ ]:
EPS = 1e-6
#slightly modified
def get_IoU(outputs, labels):
    outputs = outputs.int()
    labels = labels.int()
    # Taken from: https://www.kaggle.com/iezepov/fast-iou-scoring-metric-in-pytorch-and-numpy
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))  # Will be zero if both are 0

    iou = (intersection + EPS) / (union + EPS)  # We smooth our devision to avoid 0/0

    # thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    # return thresholded.mean()  # Or thresholded.mean() if you are interested in average across the batch
    return iou.mean()

In [ ]:
## Training model

import torch
import torch.nn as nn
from torch.nn import init
import functools
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from model import ModelClass


X = np.load("/content/drive/MyDrive/Images.npy")
Y = np.load("/content/drive/MyDrive/Masks.npy")
X=X.astype('float32')
# print(X.dtype)
# print(np.mean(X).dtype)
mean = np.mean(X)  # mean for data centering
std = np.std(X)  # std for data normalization

X -= mean  
X /= std

Y = np.where(Y > 1, 1, 0)

X_train=X[:1900]
Y_train=Y[:1900]
X_test=X[1900:]
Y_test=Y[1900:]


model=ModelClass().cuda()

lr = 3.00000002e-03 # 0.1
criterion = nn.BCELoss() #nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.8, nesterov=True, weight_decay=0.0003)

is_train = True
is_pretrain = False

total_epoch = 25
if is_train is True:

  if is_pretrain == True:
    model.load_state_dict((torch.load('/content/drive/MyDrive/Unet_resnet4.pkl')))
  
    
  
  for epoch in range(total_epoch):
      model.train()
      lossavg=0
      tot=0
      for inp,msk in zip(X_train,Y_train):
          # inp.astype(np.float32)
          # msk.astype(np.int32)


          input = Variable(torch.from_numpy(inp).type(torch.float32).cuda())
          
          mask = Variable(torch.from_numpy(msk).type(torch.float32).cuda())

          # Forward + Backward + Optimize
          optimizer.zero_grad()
          outputs = model(input.permute(2,0,1).unsqueeze(0))
          # cost=get_IoU(outputs, mask.permute(2,0,1).unsqueeze(0))
          
          # loss = Variable(cost.data, requires_grad=True)
          loss = dice_loss(outputs, mask.permute(2,0,1).unsqueeze(0)) #
          # loss = criterion(outputs, mask.permute(2,0,1).unsqueeze(0))
          loss.backward()
          # print(loss)
          optimizer.step()
          
          lossavg +=loss.item()
          tot+=1
          
      aver=lossavg/tot
      print(" ")
      
      
      print("Epoch [%d/%d],  AVERAGELoss: %.4f" %(epoch+1,total_epoch,aver))
      print('evaluate test set:')
      
      # # Decaying Learning Rate
      if (epoch+1) / float(total_epoch) == 0.3 or (epoch+1) / float(total_epoch) == 0.6 or (epoch+1) / float(total_epoch) == 0.9:
          lr /= 10
          print('reset learning rate to:', lr)
          for param_group in optimizer.param_groups:
              param_group['lr'] = lr
              print(param_group['lr'])
          # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
          # optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=0.0001)
  # Save the Model
  torch.save(model.state_dict(), '/content/drive/My Drive/Unet_resnet4.pkl')


In [ ]:
X = np.load("/content/drive/MyDrive/Images.npy")
Y = np.load("/content/drive/MyDrive/Masks.npy")
X=X.astype('float32')
# print(X.dtype)
# print(np.mean(X).dtype)
mean = np.mean(X)  # mean for data centering
std = np.std(X)  # std for data normalization

X -= mean  
X /= std

Y = np.where(Y > 1, 1, 0)

X_train=X[:1900]
Y_train=Y[:1900]
X_test=X[1900:]
Y_test=Y[1900:]

In [ ]:
loss_total=0
for img,mask in zip(X_test,Y_test):
  input = Variable(torch.from_numpy(img).type(torch.float32))
          
  msk = Variable(torch.from_numpy(mask).type(torch.float32))
  device = torch.device("cpu")
  model=ModelClass().to(device)
  
  model.load_state_dict(torch.load("/content/drive/MyDrive/last_model_full.pkl",map_location=torch.device('cpu')))
  model.eval()

  
  outputs = model(input.permute(2,0,1).unsqueeze(0))
  
  loss = get_IoU(outputs, msk.permute(2,0,1).unsqueeze(0)) 
  print(loss)
  loss_total +=loss
print("Average IOU score for test set: ",loss_total/100)


tensor(0.4034)
tensor(0.3488)
tensor(0.2079)
tensor(0.0534)
tensor(0.2041)
tensor(0.1884)
tensor(0.0596)
tensor(0.3563)
tensor(0.2936)
tensor(0.0553)
tensor(0.3366)
tensor(0.2195)
tensor(0.0946)
tensor(0.2431)
tensor(0.3837)
tensor(0.1376)
tensor(0.1731)
tensor(0.1423)
tensor(0.2435)
tensor(0.2585)
tensor(0.2244)
tensor(0.2899)
tensor(0.3953)
tensor(0.3719)
tensor(0.3680)
tensor(0.1416)
tensor(0.2625)
tensor(0.3055)
tensor(0.2545)
tensor(0.3248)
tensor(0.2506)
tensor(0.0789)
tensor(0.2392)
tensor(0.0789)
tensor(0.3405)
tensor(0.2000)
tensor(0.2510)
tensor(0.2901)
tensor(0.1691)
tensor(0.3762)
tensor(0.2664)
tensor(0.4619)
tensor(0.0841)
tensor(0.0984)
tensor(0.1452)
tensor(0.5165)
tensor(0.0754)
tensor(0.1222)
tensor(0.1062)
tensor(0.5633)
tensor(0.1343)
tensor(0.1413)
tensor(0.1455)
tensor(0.4936)
tensor(0.0480)
tensor(0.1651)
tensor(0.3602)
tensor(0.2238)
tensor(0.0669)
tensor(0.1489)
tensor(0.0633)
tensor(0.3408)
tensor(0.0478)
tensor(0.3447)
tensor(0.2195)
tensor(0.3719)
tensor(0.1